# Machine learning for genetic data

## Introduction

The goal of this practical session is to manipulate high-dimensional, low sample-size data that is typical of many genetic applications.

Here we will work with GWAS data from _Arabidopsis thaliana_, which is a plant model organism (https://upload.wikimedia.org/wikipedia/commons/6/6f/Arabidopsis_thaliana.jpg).

The genotypes are hence described by **Single Nucleotide Polymorphisms, or SNPs**. Our goal will be to use this data to identify regions of the genome that can be linked with various growth and flowering traits (**phenotypes**).

In [ ]:
%pylab inline 
# imports matplotlib as plt and numpy as np

In [ ]:
plt.rc('font', **{'size': 16}) # font size for text on plots

## Data description

* `data/athaliana_small.X.txt` is the design matrix. As many rows as samples, as many columns as SNPs
* the SNPs are given (in order) in `data/athaliana_small.snps.txt`. 
* the samples are given (in order) in `data/athaliana.samples.txt`.

* the transformed phenotypes are given in `data/athaliana.4W.pheno` and `data/athaliana.2W.pheno`. The first column is the sample's ID, and the second the phenotype.

* `data/athaliana.candidates.txt` contains a list of _A. thaliana_ genes known or strongly suspected to be associated with flowering times.

* the feature network is in `data/athaliana_small.W.txt`. It has been saved as 3 arrays, corresponding to the row, col, and data attributes of a [scipy.sparse coo_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html).

## Loading the data
We will start by working without the feature network, on the 2W phenotype.

In [ ]:
# Load the SNP names
with open('data/athaliana_small.snps.txt') as f:
    snp_names = f.readline().split()
    f.close()
print(len(snp_names))

In [ ]:
# Load the design matrix -- this can take time!
X = np.loadtxt('data/athaliana_small.X.txt',  # file names
               dtype = 'int') # values are integers

__Q: How many samples are there in the data? How many SNPs are there?__

In [ ]:
# Answer
X.shape

We have 1307 samples and 9 419 SNPs.

In [ ]:
p = X.shape[1]

#### Load the sample names

In [ ]:
samples = list(np.loadtxt('data/athaliana.samples.txt', # file names
                         dtype = int)) # values are integers
print(len(samples))

#### Load the 2W phenotype data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/athaliana.2W.pheno', # file name
                 header = None, # columns have no header
                 delim_whitespace = True) # columns are separated by white space

The 2W phenotype is not available for all samples.

In [ ]:
# Create vector of sample IDs
samples_with_phenotype = list(df[0])
print(len(samples_with_phenotype), "samples have a phenotype")

# Create vector of phenotypes
y_2W = np.array(df[1])

We need to restrict X to the samples with a 2W phenotype, in correct order

In [ ]:
X_2W = X[np.array([samples.index(sample_id) \
                   for sample_id in samples_with_phenotype]), :]

__Q: How many samples do we have now? And how many SNPs? Does this make the task of biomarker detection simpler or harder?__

In [ ]:
print(X_2W.shape)

__A:__ There are the same number of features but (way) fewer samples: the statistical power has decreased! And so the task of detecting explanatory SNPs is harder.

In [ ]:
# You can delete X now if you want, to free space
del X

#### Load the list of candidate genes

In [ ]:
with open('data/athaliana.candidates.txt') as f:
    candidate_genes = f.readline().split()
    f.close()

#### Load the snps to gene mapping

In [ ]:
genes_by_snp = {} # key: SNP, value = [genes in/near which this SNP is]
with open('data/athaliana.snps_by_gene.txt') as f:
    for line in f:
        ls = line.split()
        gene_id = ls[0]
        for snp_id in ls[1:]:
            if not snp_id in genes_by_snp:
                genes_by_snp[snp_id] = []
            genes_by_snp[snp_id].append(gene_id) 

### Split the data in a train and test set

In machine learning, we always split the data into a *train* set, which serves to fit the model, and a *test* set, which serves to measure the model's performance.

__Q: Do you remember why? What happens if we do both the training and testing on the same data?__

__A:__ Evaluating a model on the same data used to fit the model favors model that overfit the data at hand, and these models have poor generalization performance.

We will set aside a test set, containing 20% of our samples, on which to evaluate the quality of our predictive models.

__Q: What problem occurs if we set a test set that is too large in proportion? What problem occurs when it is set too small?__

__A:__ If the proportion of samples in the test set is too large, the training set becomes too small to be able to fit the model correctly. If it is too small, the measure of performance of the model will be prone to too much variability.

In [ ]:
from sklearn import model_selection

In [ ]:
X_2W_tr, X_2W_te, y_2W_tr, y_2W_te = \
    model_selection.train_test_split(X_2W, y_2W, test_size = 0.2, random_state = 17)
print(X_2W_tr.shape, X_2W_te.shape)

## Visualize the phenotype

In [ ]:
h = plt.hist(y_2W_tr, bins = 30)

Visualize the genotype's correlation structure

In [ ]:
import seaborn as sn
sigma = pd.DataFrame(X_2W_tr).corr()

fig, ax = plt.subplots(1, 2)
ax[0].imshow(sigma.iloc[0:1000, 0:1000])
ax[1].imshow(sigma.iloc[72:120, 72:120])
plt.show()

__Q: What observation can you make about the phenotype and genotype?__

__A:__ The phenotype has an unimodal distribution (only one peak), but with a few outlying values.

The genotype has a correlation structure which displays "blocks" of high correlation, called *Linkage Disequilibrium* (LD) blocks.

## T-test

Let us start by running a statistical test for association of each SNP feature with the phenotype.

In [ ]:
import statsmodels.api as sm

### T-test on a single SNP
We will perform a linear regression on a single SNP and test whether this SNP has an effect on the phenotype.

In [ ]:
est = sm.regression.linear_model.OLS(y_2W_tr, sm.add_constant(X_2W_tr[:, 0])).fit()
print(est.summary())

__Q: In the previous table, where is the p-value of the T-test? What can you conclude about the effect of the first SNP on the phenotype?__

__A:__ The p-value is 0.310: we cannot reject $\mathcal{H}_0$, and so the first SNP is estimated to have no effect on the phenotype.

We can also visualize the linear regression:

In [ ]:
sn.regplot(x = 'x', y = 'y', data = pd.DataFrame({'x': X_2W_tr[:, 0], 'y': y_2W_tr})).set(xlim = (-0.1, 1.1))

### T-test on all SNPs

In [ ]:
pvalues = []
for snp_idx in range(p):
    # only look a the column corresponding at that SNP
    X_snp = X_2W_tr[:, snp_idx]
    # run a linear regression (with bias) between the phenotype and this SNP
    X_snp = sm.add_constant(X_snp)
    est = sm.regression.linear_model.OLS(y_2W_tr, X_snp)
    est2 = est.fit()
    # get the p-value from the model 
    pvalues.append(est2.pvalues[1])
pvalues = np.array(pvalues)

### Manhattan plot

The common way to visualize such results is by using a Manhattan plot: we will plot all SNPs on the x-axis, and on the y-axis we'll have the opposite of the log base 10 of the p-value. The lower the p-value, the higher the corresponding marker. 

We will also add a horizontal line that corresponds to the _threshold for significance_. Because we are testing multiple hypotheses, we need to lower our threshold accordingly. We will use __Bonferroni correction__ and divide the significance threshold (say, alpha=0.05) by the number of tests, that is, the number of SNPs p.

In [ ]:
plt.scatter(range(p), # x = SNP position
            -np.log10(pvalues)) # y = -log10 p-value 

# significance threshold according to Bonferroni correction
t = -np.log10(0.05 / p)
plt.plot([0, p], [t, t])

# plot labels
plt.xlabel("feature")
plt.ylabel("-log10 p-value")
plt.xlim([0, p])

__Q: What do you observe? Are any SNPs significantly associated with the phenotype? 
Use data/athaliana.snps_by_gene.txt and data/athaliana.candidates.txt to check whether this matches a priori information.__

In [ ]:
# Answer

thresh = 0.05 / p # significance threshold set using the Bonferroni correction

for snp_idx in np.where(pvalues < thresh)[0]:
    print(("%.2e" % pvalues[snp_idx]), snp_names[snp_idx])
    for gene_id in genes_by_snp[snp_names[snp_idx]]:
        if gene_id in candidate_genes:
            print("\t in/near candidate gene %s" % gene_id)

__A:__ Chr5_3185806_C is just at significance threshold with a p-value of 2x10^-6. It is in or near gene [AT5G10140](https://www.arabidopsis.org/servlets/TairObject?accession=locus:2184118), which is known to play a role in flowering.

## Linear regression 

In [ ]:
from sklearn import linear_model

In [ ]:
model_lr = linear_model.LinearRegression(fit_intercept = True)
model_lr.fit(X_2W_tr, y_2W_tr)

In [ ]:
plt.figure(figsize = (12, 5))
plt.scatter(range(p), # x = SNP position
            model_lr.coef_, # y = regression weights
            s = 10)  # point size

plt.xlabel("SNP")
plt.ylabel("regression weight")
plt.xlim([0, p])

__Q: What do you observe? How can you interpret these results? Do any of the SNPs strike you as having a strong influence on the phenotype?__

__A:__ The following SNPs are the ones with the ten highest weights (in absolute value). They are all near candidate genes.

In [ ]:
highest_weights = np.abs(model_lr.coef_)
highest_weights.sort()
highest_weights = highest_weights[-10:]

for w in highest_weights:
    for snp_idx in np.where(model_lr.coef_ == w)[0]:
        print(w, snp_names[snp_idx])
        for gene_id in genes_by_snp[snp_names[snp_idx]]:
            if gene_id in candidate_genes:
                print("\t in/near candidate gene %s" % gene_id)

### Model predictive power

In this section, we measure the performance of our model on the test dataset.

We will now look at the predictive power of the lasso estimated model.

__Q: What is the definition of the variance explained? You may use the [scikit learn documentation](https://sklearn.org/modules/classes.html#sklearn-metrics-metrics). What values can this metric take? and to what cases do the extreme values correspond to?__

__A:__ The explained variance is $1 - \frac{\text{variance of the residuals}}{\text{variance of the output}}$. It is equal to one in the case of a perfect fit (the residuals are all equal to zero, which can only happen with overfitting, and is therefore not desirable). A higher value means the genotype explains a large portion of the variability of the phenotype, which is good. It can be negative (in case of arbitrarily poor fits).

In [ ]:
from sklearn import metrics

In [ ]:
y_2W_lr_pred = model_lr.predict(X_2W_te)

print("Percentage of variance explained (using all SNPs): %.2f" % \
    metrics.explained_variance_score(y_2W_te, y_2W_lr_pred))

In [ ]:
plt.figure(figsize = (5, 5))
plt.scatter(y_2W_te, y_2W_lr_pred)

plt.xlabel("true phenotype")
plt.ylabel("prediction")
plt.xlim([np.min(y_2W_te) - 5, np.max(y_2W_te) + 5])
plt.ylim([np.min(y_2W_te) - 5, np.max(y_2W_te) + 5])
plt.axline(xy1 = [0, 0], slope = 1, c = "black")

## Lasso

Define lasso model

In [ ]:
lasso = linear_model.Lasso(fit_intercept = True, max_iter = 6000)

Define cross-validation grid search and learn lasso with cross-validation.

In [ ]:
alphas = np.logspace(-4., 1., num = 20)
model_l1 = model_selection.GridSearchCV(lasso, param_grid = {'alpha': alphas}, 
                                        scoring = 'explained_variance')
model_l1.fit(X_2W_tr, y_2W_tr)

In [ ]:
plt.figure(figsize = (6, 4))
plt.scatter(range(p), # x = SNP position
            model_l1.best_estimator_.coef_)  # y = regression weights

plt.xlabel("SNP")
plt.ylabel("lasso regression weight")
plt.xlim([0, p])

__Q: How can you interpret these results? How many SNPs contribute to explaining the phenotype?__

In [ ]:
print("%d SNPs selected" % \
    np.nonzero(model_l1.best_estimator_.coef_)[0].shape)

__A:__ Only 167 SNPs have been estimated to have non-zero weights. This means that our model selects 167 explanatory SNPs (out of 9419!)

__Q: How many of theses SNPS belong to the list of *candidate SNPs*? Complete the two missing lines two to compute the number of genes hit that belong to candidate genes and the number of snps that these genes correpond to.__

In [ ]:
# Answer:
candidate_genes_hit = set([])
num_snps_in_candidate_genes = 0
for snp_idx in np.nonzero(model_l1.best_estimator_.coef_)[0]:
    for gene_id in genes_by_snp[snp_names[snp_idx]]:
        if gene_id in candidate_genes:
            candidate_genes_hit.add(gene_id)
            num_snps_in_candidate_genes += 1


print("Of which %d are in %d candidate genes" % (num_snps_in_candidate_genes, 
                                                          len(candidate_genes_hit)))

__A:__ Out of the 167 selected SNPs, 176 SNPs (counted with multiplicity) are "close" to candidate genes.

### Predictive power 

In [ ]:
y_2W_l1_pred = model_l1.best_estimator_.predict(X_2W_te)

print("Percentage of variance explained (using %d SNPs): %.2f" % \
     (np.nonzero(model_l1.best_estimator_.coef_)[0].shape[0], 
      metrics.explained_variance_score(y_2W_te, y_2W_l1_pred)))

__Q: How does the lasso compare with the OLS (linear regression) in terms of variance explained? What is the advantage of the lasso model for generating biological hypotheses?__

__A:__ Compared to unpenalized linear regression (also called OLS), the lasso has a similar variance explained, while using way fewer SNPs. So it is a better model: we need fewer mutations to explain the phenotype.

Comparing true and predicted phenotypes

In [ ]:
plt.figure(figsize = (5, 5))
plt.scatter(y_2W_te, y_2W_l1_pred)

plt.xlabel("true phenotype")
plt.ylabel("prediction")
plt.xlim([np.min(y_2W_te) - 0.05, np.max(y_2W_te) + 0.05])
plt.ylim([np.min(y_2W_te) - 0.05, np.max(y_2W_te) + 0.05])
plt.axline(xy1 = [0, 0], slope = 1, c = "black")